<a href="https://colab.research.google.com/github/SalikFillah/Topic-Modelling/blob/main/Latent_Dirichlet_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalasi Modul

In [1]:
!pip install nltk
!pip install Sastrawi
!pip install regex
!pip install unidecode
!pip install textblob
!pip install tqdm
!pip install scapy
!pip install python-crfsuite
!pip install gensim
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Modul

In [2]:
import pandas as pd
import numpy as np
import string
import nltk

# modul stemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stemmer = StemmerFactory().create_stemmer()

# modul preprocessing
nltk.download('punkt')
import re
from unidecode import unidecode
from html import unescape
from textblob import TextBlob
from tqdm import tqdm

# modul lemma & pos-tag
import spacy
from spacy.lang.id import Indonesian
from nltk.tag import CRFTagger
ct = CRFTagger()

# modul LDA
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

# modul DRT
from sklearn.manifold import TSNE

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Load Data

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/SalikFillah/Topic-Modelling/main/anies_baswedan_2024.csv')
df.head()

,title,href,body,date
0,"ANIES BASWEDAN INDONESIA on Instagram: ""Imam B...",https://www.instagram.com/p/CHKVWZrnYOT/,"940 likes, 43 comments - ANIES BASWEDAN INDONE...",2023-01-02
1,"BuddyKu Headlines on Instagram: ""Buddies! Baka...",https://www.instagram.com/p/Cn0MrvFPJdB/,Bakal Calon Presiden (Bacapres) 2024 dari Part...,2023-01-02
2,"SINDOnews on Instagram: ""Meski memiliki elekta...",https://www.instagram.com/p/CeAjzwnP0wi/,"309 likes, 29 comments - SINDOnews (@sindonews...",2023-01-02
3,"ANIES BASWEDAN INDONESIA on Instagram: ""Denger...",https://www.instagram.com/p/ClEVrd9IOEe/,"1,286 likes, 70 comments - ANIES BASWEDAN INDO...",2023-01-02
4,"ANIES BASWEDAN on Instagram: ""Calon Presiden I...",https://www.instagram.com/p/Cm6c9HFr7Cu/,"9 likes, 0 comments - ANIES BASWEDAN (@aniesra...",2023-01-02


## Preprocessing Data

### Load Slang atau Singkatan
Modifikasi sesuka hati jika sekiranya masih terdapat singkatan yang perlu diubah.

In [4]:
import urllib.request

# membaca data slang dari URL Github
url = 'https://raw.githubusercontent.com/taudataid/eLearning/master/data/slang.dic'
response = urllib.request.urlopen(url)
slang_id = eval(response.read())

# mencetak dictionary dalam format yang diinginkan
print("{")
for key, value in slang_id.items():
    print(f"    \"{key}\": \"{value}\",")
print("}")

{
    "1pun": "satupun",
    "7an": "tujuan",
    "Dr.": "doktor",
    "dr.": "dokter",
    "drg.": "dokter gigi",
    "Jkt": "Jakarta",
    "Jkw": "jokowi",
    "Napza": "narkoba psikotropika dan zat adiktif",
    "Nasihat": "nasehat",
    "ababil": "abg labil",
    "abis": "habis",
    "acc": "accord",
    "accord": "sesuai",
    "ad": "ada",
    "adl": "adalah",
    "adlah": "adalah",
    "adlh": "adalah",
    "administ": "administrasi",
    "adoh": "aduh",
    "afaik": "as far as i know",
    "agma": "agama",
    "aha": "tertawa",
    "ahaha": "haha",
    "ahiok": "ahok",
    "mehong": "mahal",
    "ahoax": "ahok",
    "ahokncc": "ahok",
    "aing": "saya",
    "aj": "saja",
    "aja": "saja",
    "ajak2": "ajak-ajak",
    "ajep-ajep": "dunia gemerlap",
    "ajj": "saja",
    "ak": "aku",
    "aka": "dikenal juga sebagai",
    "akherat": "akhirat",
    "akhirx": "akhirnya",
    "akika": "aku",
    "akko": "aku",
    "akkoh": "aku",
    "akku": "aku",
    "akn": "akan",
    "aktifis

### Load Stopword
Modifikasi sesuka hati jika sekiranya masih terdapat kata yang seharusnya dihilangkan.

In [5]:
import requests
import re

url = "https://raw.githubusercontent.com/taudataid/eLearning/master/data/stopwords_id.txt"
response = requests.get(url)
stopwords = set(re.findall(r'\w+', response.text))

# Tambahkan set tambahan ke dalam set stopwords
tambahan = set(['a', 'akan', 'atas', 
                'b', 'bisa', 'bahwa', 'bacapres', 'banyak',
                'c', 'comment', 'comments',
                'd', 'dari', 'di', 'diri',
                'e', 
                'f', 'followers', 'following', 'front', 
                'g', 
                'h',
                'i', 'instagram', 'inilahcom',
                'j', 'jateng', 'jalan', 
                'k',
                'l', 'like', 'likes', 'lalu',
                'm', 'menjadi', 'mulai', 'makin', 'meski',
                'n', 
                'o', 
                'p', 'photos', 'posts', 'punya',
                'q', 
                'r', 'rizieq',
                's', 'sebagai', 'saja', 'sama', 'orang', 'salah', 'selalu', 'satu', 'sindonews', 'saling', 'see',
                't', 'tribun',
                'u', 
                'v', 
                'w', 
                'x', 
                'y',
                'z',])
stopwords = stopwords.union(tambahan)


# Meletakkan setiap kata dalam kurung kurawal dan memberikan tanda kutip
stopwords_id = "{" + ", ".join([f'"{word}"' for word in stopwords]) + "}"

print(len(stopwords))

791


### NLP (Natural Language Preprocessing)
Membersihkan data teks dari karakter-karakter yang tidak diperlukan serta menangani stopword dan slang atau singkatan yang sudah di load sebelumnya dan lain sebagainya.

Note : setiap platform media sosial memiliki cara pembersihan data nya masing-masing (modifikasi sesuka hati).

In [6]:
def cleanbody(text):
    
    # menghapus url
    url_pattern = re.compile(r'(\w+:\/\/\S+)')
    text = url_pattern.sub(' ', text)
    
    # menghapus hashtag
    hashtag_pattern = re.compile(r'#\w+\b')
    text = hashtag_pattern.sub(' ', text)
    
    # menghapus nama pengguna instagram (memuat "@")
    username_pattern = re.compile(r'@\w+\b\s*')
    text = username_pattern.sub(' ', text)
    
    # menghapus angka
    text = re.sub(r'\b\d+\b|[^\w\s]', '', text)
    
    # mengahpus simbol
    symbol_pattern = re.compile(r'[^\w\s]+')
    text = symbol_pattern.sub(' ', text)
    
    # menghapus karakter yang tidak diperlukan (tergantung media sosial)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\r', ' ', text)
    text = re.sub(r'&\w+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # menangani huruf kapital dan spasi
    text = unidecode(unescape(text.lower().strip()))
    
    # menangani slang atau singkatan 
    Tokens = TextBlob(text).words
    for i,t in enumerate(Tokens):
      if t in slang_id.keys():
        Tokens[i] = slang_id[t]
        
    # menangani stopword
    text = ' '.join([t for t in Tokens if str(t) not in stopwords_id and len(t)>2])
    
    # stemming
    text = stemmer.stem(text)
    
    
    return text

In [7]:
# aplikasikan fungsi ke dalam kolom variabel baru
df['clean_body'] = ''
for idx, post in tqdm(df.iterrows()):
    df.at[idx, 'clean_body'] = cleanbody(post.body)

34it [00:04,  7.28it/s]


In [8]:
df.head()

,title,href,body,date,clean_body
0,"ANIES BASWEDAN INDONESIA on Instagram: ""Imam B...",https://www.instagram.com/p/CHKVWZrnYOT/,"940 likes, 43 comments - ANIES BASWEDAN INDONE...",2023-01-02,anies baswedan indonesia imam bela islam front...
1,"BuddyKu Headlines on Instagram: ""Buddies! Baka...",https://www.instagram.com/p/Cn0MrvFPJdB/,Bakal Calon Presiden (Bacapres) 2024 dari Part...,2023-01-02,calon presiden partai nasdem anies baswedan ha...
2,"SINDOnews on Instagram: ""Meski memiliki elekta...",https://www.instagram.com/p/CeAjzwnP0wi/,"309 likes, 29 comments - SINDOnews (@sindonews...",2023-01-02,milik elektabilitas survei anies baswedan lem ...
3,"ANIES BASWEDAN INDONESIA on Instagram: ""Denger...",https://www.instagram.com/p/ClEVrd9IOEe/,"1,286 likes, 70 comments - ANIES BASWEDAN INDO...",2023-01-02,anies baswedan indonesia dengar
4,"ANIES BASWEDAN on Instagram: ""Calon Presiden I...",https://www.instagram.com/p/Cm6c9HFr7Cu/,"9 likes, 0 comments - ANIES BASWEDAN (@aniesra...",2023-01-02,anies baswedan calon presiden indonesia


### Lemmatisasi & Pos-Tag
Bagian preprocessing yang paling vital atau krusial untuk proses LDA dari yang lainnya, yaitu identifikasi kata dasar serta pengelompokkan kata berdasarkan kategori kata tersebut.

In [9]:
# lemma & pos-tag bahasa indonesia
nlp_id = Indonesian()
!wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger
ct.set_model_file('data/all_indo_man_tag_corpus_model.crf.tagger')

--2023-05-08 02:39:16--  https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1722780 (1.6M) [application/octet-stream]
Saving to: ‘data/all_indo_man_tag_corpus_model.crf.tagger’

all_indo_man_tag_co 100%[===================>]   1.64M  --.-KB/s    in 0.06s   

2023-05-08 02:39:16 (28.8 MB/s) - ‘data/all_indo_man_tag_corpus_model.crf.tagger’ saved [1722780/1722780]



In [10]:
def NLPfilter(t, filters):

  # lemmatisasi
  tokens = nlp_id(t)

  # tokenisasi
  tokens = [str(k) for k in tokens if len(k)>2]

  # pos-tag
  hasil = ct.tag_sents([tokens])
  
  return [k[0] for k in hasil[0] if k[1] in filters]

  Note :
- NN : kata benda tunggal (meja, buku, kucing, cinta, ...)
- NNP : kata benda tunggal khusus (indonesia, google, nike, tokyo, ...)
- NNS : kata benda jamak (buku-buku, meja-meja, ...)
- NNPS : kata benda jamak khusus (beatles, avengers, simpsons, ...)
- JJ : kata sifat (marah, tinggi, besar, indah, ...)

In [11]:
# ambil variabel kolom hasil preprocessing
data = df['clean_body'].values

# pilih kategori kata 
filters = set(['NN', 'NNP', 'NNS', 'NNPS', 'JJ'])

# aplikasikan fungsi ke dalam dataframe baru
data_postTag = []
for i, d in tqdm(enumerate(data)):
    data_postTag.append(NLPfilter(d,filters))

' '.join(data_postTag[0])

34it [00:00, 1007.85it/s]


'anies baswedan indonesia imam bela islam front bela islam habib shihab umum indones anies baswedan indonesia imam bela islam front bela islam habib shihab umum indonesia'

In [12]:
# tokenisasi data kembali
data = [d for d in data_postTag if d]

## LDA (Latent Dirichlet Allocation)


In [13]:
# membuat representasi dictionary dari dokumen

# membuang token yang langka dan umum
dictionary_t = Dictionary(data)
dictionary_t.filter_extremes(no_below=2, no_above=0.90)

# membuat dictionary dan corpus yang diperlukan topic modelling
corpus_t = [dictionary_t.doc2bow(doc) for doc in data]
corpus_t = [t for t in corpus_t if t] # membuang corpus atau dokumen yang kosong

print('Number of unique tokens: %d' % len(dictionary_t))
print('Number of documents: %d' % len(corpus_t))
print(corpus_t[:1])

Number of unique tokens: 31
Number of documents: 32
[[(0, 2), (1, 2), (2, 3), (3, 2)]]


Pembuatan dataframe hasil keluaran dari algoritma LDA (Latent Dhiriclet Allocation).

In [14]:
def format_topics_sentences(ldamodel, corpus, texts, dates):
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([
                    sent_topics_df,                   
                pd.DataFrame([[int(topic_num), round(prop_topic, 4), topic_keywords]],
                             columns=["Dominant_Topic", "Perc_Contribution", "Topic_Keywords"])],
                    ignore_index=True,
                )
            else:
                break
    sent_topics_df.columns = ["Dominant_Topic", "Perc_Contribution", "Topic_Keywords"]

    # Add original text to the end of the output
    contents = pd.Series(texts)

    sent_topics_df = pd.concat([sent_topics_df, contents, pd.Series(dates)], axis=1)
    return sent_topics_df

Model DRT (Dimensional Reduction Technique).

In [15]:
def tsne_analysis(ldamodel, corpus):
    topic_weights = []
    for i, row_list in enumerate(ldamodel[corpus]):
        topic_weights.append([w for i, w in row_list])

    # Array of topic weights
    df_topics = pd.DataFrame(topic_weights).fillna(0).values

    # Keep the well separated points (optional)
    # arr = arr[np.amax(arr, axis=1) > 0.35]

    # Dominant topic number in each doc
    topic_nums = np.argmax(df_topics, axis=1)

    # tSNE Dimension Reduction
    try:
        tsne_model = TSNE(
            n_components=2, verbose=1, random_state=0, angle=0.99, init="pca"
        )
        tsne_lda = tsne_model.fit_transform(df_topics)
    except:
        print("TSNE_ANALYSIS WENT WRONG, PLEASE RE-CHECK YOUR BANK DATASET")
        return (topic_nums, None)

    return (topic_nums, tsne_lda)

Penggabungan dataframe sedemikian sehingga data ini lah yang nantinya di aplikasikan ke dalah dashboard LDA.

In [16]:
def lda_analysis(df):
    
    docs = list(df["clean_body"].values)

    punctuations = string.punctuation

    processed_docs = data
    print("Jumlah corpus atau dokumen", len(processed_docs))
    if len(processed_docs) < 11:
        print("INSUFFICIENT DOCS TO RUN LINEAR DISCRIMINANT ANALYSIS")
        return (None, None, None, None)

    print("Jumlah BoW (Bag of Words) corpus", len(corpus_t))
    print("Jumlah dictionary", len(list(dictionary_t.keys())))
    if len(list(dictionary_t.keys())) < 1:
        print("INSUFFICIENT DICTS TO RUN LINEAR DISCRIMINANT ANALYSIS")
        return (None, None, None, None)

    lda_model = LdaModel(
        corpus_t, num_topics=5, id2word=dictionary_t, passes=10
    )

    df_topic_sents_keywords = format_topics_sentences(
        ldamodel=lda_model,
        corpus=corpus_t,
        texts=docs,
        dates=list(df["date"].values),
    )
    print("Jumlah data", len(df_topic_sents_keywords))
    print("Data", df_topic_sents_keywords.head())
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = [
        "Document_No",
        "Dominant_Topic",
        "Topic_Perc_Contrib",
        "Keywords",
        "Text",
        "Date",
    ]

    print("Hasil DRT")
    topic_num, tsne_lda = tsne_analysis(lda_model, corpus_t)

    return (tsne_lda, lda_model, topic_num, df_dominant_topic)

In [17]:
# aplikasikan fungsi dan simpan hasilnya ke dalam beberapa variabel berikut
tsne_lda, lda_model, topic_num, df_dominant_topic = lda_analysis(df)

Jumlah corpus atau dokumen 34
Jumlah BoW (Bag of Words) corpus 32
Jumlah dictionary 31
Jumlah data 34
Data    Dominant_Topic  Perc_Contribution  \
0             0.0             0.9197   
1             0.0             0.9198   
2             3.0             0.7473   
3             0.0             0.7989   
4             0.0             0.8662   

                                      Topic_Keywords  \
0  anies, baswedan, indonesia, calon, presiden, p...   
1  anies, baswedan, indonesia, calon, presiden, p...   
2  gubernur, anies, baswedan, daerah, khusus, ibu...   
3  anies, baswedan, indonesia, calon, presiden, p...   
4  anies, baswedan, indonesia, calon, presiden, p...   

                                                   0           1  
0  anies baswedan indonesia imam bela islam front...  2023-01-02  
1  calon presiden partai nasdem anies baswedan ha...  2023-01-02  
2  milik elektabilitas survei anies baswedan lem ...  2023-01-02  
3                    anies baswedan indonesia d

In [18]:
# data untuk pembuatan dashboard
df_dominant_topic.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Date
0,0,0.0,0.9197,"anies, baswedan, indonesia, calon, presiden, p...",anies baswedan indonesia imam bela islam front...,2023-01-02
1,1,0.0,0.9198,"anies, baswedan, indonesia, calon, presiden, p...",calon presiden partai nasdem anies baswedan ha...,2023-01-02
2,2,3.0,0.7473,"gubernur, anies, baswedan, daerah, khusus, ibu...",milik elektabilitas survei anies baswedan lem ...,2023-01-02
3,3,0.0,0.7989,"anies, baswedan, indonesia, calon, presiden, p...",anies baswedan indonesia dengar,2023-01-02
4,4,0.0,0.8662,"anies, baswedan, indonesia, calon, presiden, p...",anies baswedan calon presiden indonesia,2023-01-02


## Referensi
1.   https://taudata.blogspot.com/2022/05/nlptm-07.html
2.   https://github.com/plotly/dash-sample-apps/blob/main/apps/dash-nlp/ldacomplaints.py
3. https://github.com/taudataid/eLearning